# Interactive clingo (using Multi Shot Solving)

The task of this notebook is to implement in clingo an interactive answer set solver, based on the ideas of the paper [1]. 

Please, read section 1, 3 and 4 of that paper before starting this project.

You can use the script `iclingo.py` as a starting point for your interactive solver.

[[1] Gebser, M., Obermeier, P., & Schaub, T. (2015). Interactive Answer Set Programming - Preliminary Report. CoRR, abs/1511.01261.](https://www.cs.uni-potsdam.de/wv/publications/DBLP_journals/corr/GebserOS15.pdf)

## Task

The main difference between our approach and the approach presented in [1] is that we do not implement directly the command `query`. Instead, we use a combination of other commands to achieve a similar functionality.

In addition, we define two new groups of commands:
* `opt-on` and `opt-off` to turn on and off optimization, and
* `show-add`, `show-remove`, `hide-add` and `hide-remove` to mantain a list of shown and hidden predicates that are used to determine what atoms are shown

The script should optionally accept a file including a sequence of commands, that should be run before starting the interaction.

In our simplest example, given the file `example/example.lp`:

In [2]:
! cat example/example.lp

dom(1..2).
1 { a(X) : dom(X) }.
b(X) :- a(X).
#show b/1.


You can run this sequence of commands:

In [4]:
! cat example/example-batch.txt

load example.lp
solve
exit


and obtain:

In [6]:
! python iclingo.py example/example-batch.txt

?- load example/example.lp

?- solve
Model: [b(2)]
SAT

?- exit


Your script should be able to simulate the approach of [1]. For this, the file `ncoloring.lp` of [1] is available at `ncoloring/ncoloring.lp`:

In [8]:
! cat ncoloring/ncoloring.lp 

#const n = 3.
#const m = 10.
color(1..n).

% Extract nodes from edges
node(X) :- edge(X,_).
node(X) :- edge(_,X).

% Generate n-coloring
1 { mark(X,C) : color(C) } 1 :- node(X).
:- edge(X,Y), mark(X,C), mark(Y,C).

% Allow aspic user to add and remove edges via externals
#external edge(X,Y) : X = 1..m-1, Y = X+1..m.

% Display n-coloring
#show mark/2.


In the directory `ncoloring` there is also a batch file simulating the commands used in the examples of [1]:

In [9]:
! cat ncoloring/ncoloring-batch.txt

load ncoloring.lp
solve
assert edge(1,2)
assert edge(1,4)
assert edge(2,3)
assert edge(3,4)
assume mark(1,1)
solve
cancel mark(1,1)
option -n 0
assume mark(1,1)
solve
cancel mark(1,1)
option -e brave
assume mark(1,1)
solve
cancel mark(1,1)
option -e cautious
assume mark(1,1)
solve
cancel mark(1,1)
option -e auto
add query1 :- mark(1,1), 1 { mark(3,2); not mark(4,2) }.
assume query1
solve
cancel query1
assert edge(2,4)
assume query1
solve
cancel query1
open edge(2,4)
assume query1
solve
cancel query1
retract edge(2,4)
assume query1
solve
cancel query1
assume not mark(2,3)
assume mark(1,1)
solve
cancel mark(1,1)
assume query1
solve
cancel query1
cancel not mark(2,3)
add #external elim(X,C) : X = 1..m, color(C).
add :- edge(X,Y), elim(X,C), mark(Y,C).
add :- edge(X,Y), mark(X,C), elim(Y,C).
assert elim(2,3)
assert elim(4,2)
add query2 :- mark(X,1), elim(X,C).
assume query2
solve
cancel query2
add query3 :- mark(X,C), elim(X,C).
assume query3
solve
cancel query3
add #minimize{ C@X : mark(X

Observe how the `query` commands are simulated by a mixture of:
* `assume` and `cancel` commands, and
* `add` commands to add rules defining new atoms `query1`, `query2`, ..., when needed.

The output generated by your script should look like this:

In [10]:
! cat ncoloring/ncoloring.out

?- load ncoloring.lp

?- solve
Model: []
SAT

?- assert edge(1,2)

?- assert edge(1,4)

?- assert edge(2,3)

?- assert edge(3,4)

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,3), mark(3,2), mark(4,3)]
SAT

?- cancel mark(1,1)

?- option -n 0

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,3), mark(3,2), mark(4,3)]
Model: [mark(1,1), mark(2,3), mark(3,1), mark(4,3)]
Model: [mark(1,1), mark(2,2), mark(3,1), mark(4,3)]
Model: [mark(1,1), mark(2,3), mark(3,1), mark(4,2)]
Model: [mark(1,1), mark(2,2), mark(3,1), mark(4,2)]
Model: [mark(1,1), mark(2,2), mark(3,3), mark(4,2)]
SAT

?- cancel mark(1,1)

?- option -e brave

?- assume mark(1,1)

?- solve
Model: [mark(1,1), mark(2,2), mark(2,3), mark(3,1), mark(3,2), mark(3,3), mark(4,2), mark(4,3)]
SAT

?- cancel mark(1,1)

?- option -e cautious

?- assume mark(1,1)

?- solve
Model: [mark(1,1)]
SAT

?- cancel mark(1,1)

?- option -e auto

?- add query1 :- mark(1,1), 1 { mark(3,2); not mark(4,2) }.

?- assume query1

?- solve
M